In [21]:
# Need only first time
#conda install -c conda-forge python-dotenv
#conda install -c conda-forge pydot

In [30]:
%matplotlib inline

import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
from pandas_profiling import ProfileReport
#import librosa
#import librosa.display
  
import sys
import pathlib
sys.path.insert(1, './fma_metadata/lib/')
import utils

plt.rcParams['figure.figsize'] = (17, 5)

In [3]:
# Load metadata and features.
genres = utils.load('fma_metadata/genres.csv')
features = utils.load('fma_metadata/features.csv')
echonest = utils.load('fma_metadata/echonest.csv')
tracks = utils.load('fma_metadata/tracks.csv')


np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, genres.shape, features.shape, echonest.shape

((106574, 52), (163, 4), (106574, 518), (13129, 249))

In [15]:
ipd.display(tracks['track'].head())
ipd.display(tracks['album'].head())
ipd.display(tracks['artist'].head())
ipd.display(tracks['set'].head())

,bit_rate,comments,composer,date_created,date_recorded,duration,favorites,genre_top,genres,genres_all,information,interest,language_code,license,listens,lyricist,number,publisher,tags,title
track_id,,,,,,,,,,,,,,,,,,,,
2,256000,0,NaN,2008-11-26 01:48:12,2008-11-26,168,2,Hip-Hop,[21],[21],NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
3,256000,0,NaN,2008-11-26 01:48:14,2008-11-26,237,1,Hip-Hop,[21],[21],NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
5,256000,0,NaN,2008-11-26 01:48:20,2008-11-26,206,6,Hip-Hop,[21],[21],NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
10,192000,0,Kurt Vile,2008-11-25 17:49:06,2008-11-26,161,178,Pop,[10],[10],NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
20,256000,0,NaN,2008-11-26 01:48:56,2008-01-01,311,0,NaN,"[76, 103]","[17, 10, 76, 103]",NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type
track_id,,,,,,,,,,,,,
2,0,2008-11-26 01:44:45,2009-01-05,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album
3,0,2008-11-26 01:44:45,2009-01-05,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album
5,0,2008-11-26 01:44:45,2009-01-05,NaN,4,1,<p></p>,6073,NaN,[],AWOL - A Way Of Life,7,Album
10,0,2008-11-26 01:45:08,2008-02-06,NaN,4,6,NaN,47632,NaN,[],Constant Hitmaker,2,Album
20,0,2008-11-26 01:45:05,2009-01-06,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,[],Niris,13,Album


,active_year_begin,active_year_end,associated_labels,bio,comments,date_created,favorites,id,latitude,location,longitude,members,name,related_projects,tags,website,wikipedia_page
track_id,,,,,,,,,,,,,,,,,
2,2006-01-01,NaT,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,[awol],http://www.AzillionRecords.blogspot.com,NaN
3,2006-01-01,NaT,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,[awol],http://www.AzillionRecords.blogspot.com,NaN
5,2006-01-01,NaT,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0,2008-11-26 01:42:32,9,1,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,[awol],http://www.AzillionRecords.blogspot.com,NaN
10,NaT,NaT,"Mexican Summer, Richie Records, Woodsist, Skul...","<p><span style=""font-family:Verdana, Geneva, A...",3,2008-11-26 01:42:55,74,6,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,"[philly, kurt vile]",http://kurtvile.com,NaN
20,1990-01-01,2011-01-01,NaN,<p>Songs written by: Nicky Cook</p>\n<p>VOCALS...,2,2008-11-26 01:42:52,10,4,51.895927,Colchester England,0.891874,Nicky Cook\n,Nicky Cook,NaN,"[instrumentals, experimental pop, post punk, e...",NaN,NaN


,split,subset
track_id,,
2,training,small
3,training,medium
5,training,small
10,training,small
20,training,large


In [51]:
#in python2 funzionava, nel 3 non possiamo farlo
#small = tracks[(tracks['set', 'subset']) <= ('small')]

small = tracks[ (tracks['set', 'subset'] == 'small') ]
small.shape

(8000, 52)

In [55]:
#in python2 funzionava, nel 3 non possiamo farlo
#medium = tracks[tracks['set', 'subset'] <= 'medium']

medium = tracks[ (tracks['set', 'subset'] == 'small') | (tracks['set', 'subset'] == 'medium') ]
medium.shape

(25000, 52)

In [10]:
import pandas as pd
d = pd.DataFrame({'nome': ['bruno','andrea','giacomo','francesco','raffa','tiziano'],
     'cognome': ['antinori','mallegni','zotti','gemignani','spina','gemignani'],
      'eta':[21,34,54,23,5,6]})
d

,nome,cognome,eta
0,bruno,antinori,21
1,andrea,mallegni,34
2,giacomo,zotti,54
3,francesco,gemignani,23
4,raffa,spina,5
5,tiziano,gemignani,6


In [20]:
d[ (d['nome']<='spina') & (d['cognome']<='spina') ]

,nome,cognome,eta
0,bruno,antinori,21
1,andrea,mallegni,34
3,francesco,gemignani,23
4,raffa,spina,5


In [17]:
trackrep = ProfileReport(tracks['track'])
trackrep.to_file(output_file='profiling/tracks/track-profiling.html')
trackrep = ProfileReport(tracks['album'])
trackrep.to_file(output_file='profiling/tracks/album-profiling.html')
trackrep = ProfileReport(tracks['artist'])
trackrep.to_file(output_file='profiling/tracks/artist-profiling.html')
trackrep = ProfileReport(tracks['set'])
trackrep.to_file(output_file='profiling/tracks/set-profiling.html')

In [11]:
genrep = ProfileReport(genres)
genrep.to_file(output_file='profiling/genres-profiling.html')

In [ ]:
echorep = ProfileReport(echonest['echonest']['audio_features'])
echorep.to_file(output_file='profiling/echonest/audio_features.html')

echorep = ProfileReport(echonest['echonest']['metadata'])
echorep.to_file(output_file='profiling/echonest/metadata.html')

echorep = ProfileReport(echonest['echonest']['ranks'])
echorep.to_file(output_file='profiling/echonest/ranks.html')

echorep = ProfileReport(echonest['echonest']['social_features'])
echorep.to_file(output_file='profiling/echonest/social_features.html')

echorep = ProfileReport(echonest['echonest']['temporal_features'])
echorep.to_file(output_file='profiling/echonest/temporal_features.html')

In [ ]:
#featrep = ProfileReport(features)
#featrep.to_file(output_file='profiling/features-profiling.html')
for col in features.columns: 
    print(col)